Trabalho 1 de Algoritmos 2 do Curso de Ciência da Computação da Universidade Federal de Minas Gerais (UFMG)
2024/1

Geometria Computacional - Problema da Galeria de Arte (Art Gallery Problem)

Solução com Triangulação de Polígono e Coloração de Vértices


Feito pelos alunos:

Gustavo Ferreira Dias

Vinicius Trindade Dias Abel

Código completo e detalhado assim como mais polígonos de teste e relatório disponível em: https://github.com/Gustavof1/Alg2-Tp1.git




1. Introdução

O problema proposto foi implementar um algoritmo para triangular um polígono, usando o método do corte de orelhas e computar o limite inferior de câmeras para vigiá-lo usando o algoritmo da coloração de vértices em um grafo. O programa deve permitir que o usuário possa interagir e analisar passo-a-passo o algoritmo, destacando as faces que já foram exploradas, ou a face que está sendo avaliada em cada iteração utilizando a biblioteca Plotly.


2. Solução

Primeiro importamos a biblioteca necessária Plotly para a visualização dos dados e criamos um array que guarda todas as etapas da triangulação para a visualização dos dados.

In [1]:
#importações
import plotly.graph_objects as go

#Guarda as etapas da triangulação
triangulos_plot = []

Em seguida inicializamos o polígono.

In [2]:
#Polígono de exemplo
polygon = [(11.738629630766809, 7.3216515351086855), (11.860933776944876, 7.723879873752594), (7.699581298045814, 9.777790376916528), (5.43849278613925, 10.247327791526914), (7.7646241541951895, 7.545410039834678), (10.59512207005173, 7.4548459416255355), (8.793638563714921, 4.844651281833649), (3.3427570620551705, 10.970680316910148), (3.257169222459197, 10.20868988148868), (-0.1532835876569152, 8.385072126053274), (-2.0294700264930725, 3.1217468148097396), (-2.0232564210891724, 5.403068688698113), (-2.719149355776608, 5.188982569612563), (-2.0808586115017533, 1.9411433571949601), (2.4004607666283846, 4.195138857699931), (2.6442698864266276, 1.7809057272970676), (5.08361873499883, 1.8030554465949535), (11.187730520032346, 5.569734688848257), (14.40893903374672, 2.865135650150478), (14.494080721400678, 9.34415570832789)]

fig = go.Figure()

x, y = zip(*polygon)
fig.add_trace(go.Scatter(visible=True, x=x + (x[0],), y=y + (y[0],), mode='lines', name='Polígono'))

fig.update_layout(title='Polígono',
                  xaxis_title='X',
                  yaxis_title='Y',)

fig.show()

Agora executamos a triangulação usando o método de corte de orelhas.

A função funciona da seguinte forma: Uma orelha de um polígono é o triângulo formado pelos vértices consecutivos u, v, w, tal que uw é uma diagonal. Nesse caso, v é chamado de ponta da orelha. O algoritmo inicia verificando quais vértices formam orelhas (quais são pontas de orelhas), para isso ele chama a função isConvex(), O(1) para os pontos vertices[i], vertices[i+1], vertices[i+2]. A função isConvex verifica se o produto vetorial entre os vetores que formam o triângulo, dado os 3 vértices que o compõem, é maior que 0, se sim, significa que o triângulo é convexo, se não, não é convexo. Caso os 3 pontos não formem um triângulo convexo, é passado para os próximos pontos da lista.

Caso os vértices formem um triângulo convexo, é testado para cada outro ponto do polígono, se ele faz parte do triângulo ou se ele está dentro do triângulo. Para verificar se um ponto está dentro do triângulo é usado a função inTriangle(), O(1), que, dado 1 ponto e 3 pontos que formam um triângulo, é chamado a função isConvex entre o ponto e cada aresta do triângulo. Este passo onde algum outro ponto do polígono está dentro do triângulo é demonstrado no gráfico com uma linha pontilhada vermelha.

Se nenhum outro ponto do polígono está nessas condições significa que o vertices[i+1] = v, ou seja, o triângulo é uma orelha e v é sua ponta. Com isso agora basta retirar ele da lista de vértices a serem computados na próxima iteração. Esse processo tem custo O(n) para cada vértice, logo custo total, no pior caso O(n²).

É criado na variável triangle uma lista de tuplas de tuplas, na qual cada tupla da lista possui 3 tuplas e cada uma dessas tuplas possuem 2 valores, X e Y que correspondem aos vértices que formam o triângulo.

Exemplo: [((p1t1x,p1t1y),(p2t1x,p2t1y),(p3t1x,p3t1y)),((p1t2x,p1t2y),(p2t2x,p2t2y),(p3t2x,p3t2y)),((p1t3x,p1t3y),(p2t3x,p2t3y),(p3t3x,p3t3y))]

E na variável triangulos_plot é armazenado também os passos onde é encontrado um ponto dentro de um triângulo analisado.

In [3]:
# Função para verificar se dados 3 pontos eles formam um triângulo convexo
def isConvex(p1, p2, p3):
  return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]) >= 0

# Função para verificar se um ponto está dentro de um triângulo
def inTriangle(pt, v1, v2, v3):
  b1 = not isConvex(pt,v1, v2)
  b2 = not isConvex(pt,v2, v3)
  b3 = not isConvex(pt,v3, v1)
  return ((b1 == b2) and (b2 == b3))

triangles = []
vertices = polygon[:]
while len(vertices) > 3:
    for i in range(len(vertices)):

        p1 = vertices[i]
        p2 = vertices[i + 1]
        p3 = vertices[i + 2]

        #se os 3 pontos analisados formam um triangulo convexo
        if isConvex(p1, p2, p3):
            orelha = True
            #para cada j, se ele não faz parte/está dentro do triângulo, vertice i+1 não é ponta da orelha
            for j in vertices:
                if not (j in (p1,p2,p3)) and inTriangle(j, p1, p2, p3):
                    orelha = False
                    triangulos_plot.append((p1, p2, p3))
                    break

            if orelha:
                triangles.append((p1, p2, p3))
                triangulos_plot.append((p1, p2, p3))
                del vertices[i + 1]
                break

#faz ultimo triangulo
triangles.append((vertices[0], vertices[1], vertices[2]))
triangulos_plot.append((vertices[0], vertices[1], vertices[2]))

print(triangles)

[((11.738629630766809, 7.3216515351086855), (11.860933776944876, 7.723879873752594), (7.699581298045814, 9.777790376916528)), ((11.738629630766809, 7.3216515351086855), (7.699581298045814, 9.777790376916528), (5.43849278613925, 10.247327791526914)), ((5.43849278613925, 10.247327791526914), (7.7646241541951895, 7.545410039834678), (10.59512207005173, 7.4548459416255355)), ((11.738629630766809, 7.3216515351086855), (5.43849278613925, 10.247327791526914), (10.59512207005173, 7.4548459416255355)), ((11.738629630766809, 7.3216515351086855), (10.59512207005173, 7.4548459416255355), (8.793638563714921, 4.844651281833649)), ((8.793638563714921, 4.844651281833649), (3.3427570620551705, 10.970680316910148), (3.257169222459197, 10.20868988148868)), ((8.793638563714921, 4.844651281833649), (3.257169222459197, 10.20868988148868), (-0.1532835876569152, 8.385072126053274)), ((-2.0294700264930725, 3.1217468148097396), (-2.0232564210891724, 5.403068688698113), (-2.719149355776608, 5.188982569612563)), 

In [4]:
fig = go.Figure()

x, y = zip(*polygon)
fig.add_trace(go.Scatter(visible=True,
                                x=x + (x[0],),
                                y=y + (y[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='blue'),
                                name='Polígono',
                                showlegend=True))

step_index = 1
x_cumulative = ()
y_cumulative = ()

for triangle in triangulos_plot:
    tx, ty = zip(*triangle)

    #se é um triangulo correto
    if triangle in triangles:
        x_cumulative += tx
        y_cumulative += ty
        fig.add_trace(go.Scatter(visible=False,
                                x=x_cumulative + (tx[0],),
                                y=y_cumulative + (ty[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='green'),
                                name='Polígono Triangulado',
                                showlegend=True))
    else:
        fig.add_trace(go.Scatter(visible=False,
                                x=x + (x[0],) + tx + (tx[0],),
                                y=y + (y[0],) + ty + (ty[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='red',dash='dot'),
                                name='Vértice dentro do Triangulo',
                                showlegend=True))
    step_index = step_index + 1

steps = []
i = 0
while i < len(fig.data):
    pular2 = False

    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
            {"title": "Passo: " + str(i)}],
    )
    step["args"][0]["visible"][i] = True
    step["args"][0]["visible"][0] = True #Polígono inicial sempre visível

    steps.append(step)
    i += 1

sliders = [dict(
    active=0,
    currentvalue={"prefix": ""},
    steps=steps
)]

fig.update_layout(title='Triangulação por Corte de Orelhas e Coloração dos vértices',
                  xaxis_title='X',
                  yaxis_title='Y',
                  sliders=sliders)

fig.show()

Agora criamos a estrutura de um grafo com a função de coloração de vértices

In [6]:
#grafo não direcionado, representação por matriz de adjacências
class Grafo:
    def __init__(self, vertices):
        self.vertices = vertices
        self.grafo = [[0 for _ in range(vertices)] for _ in range (vertices)]

    def connectVertices(self, a, b):
        self.grafo[a][b] = 1
        self.grafo[b][a] = 1

    #checa se alguem conectado a v tem a mesma cor
    def conectadoComMesmaCor(self, vertice, array_cores, cor):
        for i in range(self.vertices):
            if self.grafo[i][vertice] == 1 and array_cores[i] == cor:
                return False
        return True

    def vertexColor(self, quantidade_cores, array_cores, vertice_atual):
        if vertice_atual == self.vertices:
            return True

        for cor in range(1, quantidade_cores + 1):
            if self.conectadoComMesmaCor(vertice_atual, array_cores, cor):
                array_cores[vertice_atual] = cor
                if self.vertexColor(quantidade_cores, array_cores, vertice_atual + 1):
                    return True
                array_cores[vertice_atual] = 0

        return False

Fazemos a transformação do polígono para grafo.

Primeiro é criado um grafo com a quantidade de vértices referente a quantidade de vértices do polígono, o grafo inicia uma matriz de adjacências de tamanho nxn para n a quantidade de vértices do polígono. Em seguida ela realiza as conexões básicas do polígono com a função connectVertices() (marca com 1 na matriz de adjacências caso o vértice que corresponde a linha se conecte com o vértice correspondente à coluna), essas conexões básicas representam o polígono inicial, onde o ponto1 conecta com o ponto2, ponto2 com ponto3 e em diante. Após isso é enumerado cada vértice e guardado em um dicionário, dando uma chave numérica para cada coordenada (x,y), essa chave representa um vértice no grafo. Em seguida é construído uma lista que guarda as conexões dos pontos dos triângulos e essa lista é executada para conectar os vértices correspondentes no grafo. Agora representamos o polígono triangulado em forma de grafo com todas as suas conexões. O custo dessa transformação no pior caso é de O(xn) para n a quantidade de vértices e x a quantidade máxima que que cada vértice tem de conexões.

Depois da transformação de polígono para grafo, executamos a função vertexColor() no grafo criado. Passamos o parâmetro 3 para a função porque sabemos que no polígono triangulado, é possível colorir seus vértices sem que haja 2 vértices conectados que possuam a mesma cor com somente 3 cores. A função vertexColor() testa todas as possibilidades de coloração de vértices até encontrar uma que satisfaça. O custo dessa função é O(3^n), para n a quantidade de vértices, porque ele testa todas as combinações possíveis de coloração com 3 cores, (isso reduz o desempenho do código para polígonos com muitos vértices).

Na variável vertex_colors é criado um dicionário que indica a cor de cada vértice.

In [7]:
#transforma em grafo para
#garantir que o polígono e os triangulos são 1 só objeto conectado

#criando grafo, tamanho = vertices que compoem poligono
g = Grafo(len(polygon))

#traduzindo conexões
#polígono é fechado
#0 no 1, 1 no 2, 2 no 3, 3 no 4, 4 no 5,...
for i in range(len(polygon)-1):
    g.connectVertices(i, i+1)
#ligação último no primeiro, n no 0
g.connectVertices(len(polygon)-1, 0)

#Transforma lista de tuplas em um dicionário,
# 0 : (coordx, coordy) , 1 : (coordx, coordy)...
dicionario_indice_vertices = {tupla_: i for i, tupla_ in enumerate(polygon)}

#triangulo tem 3 vertices
#transforma em conexão, v1->v2, v2->v3, v3->v1
conexoes = []

for triangle in triangles:
    conexoes.append((triangle[0],triangle[1]))
    conexoes.append((triangle[1],triangle[2]))
    conexoes.append((triangle[2],triangle[0]))

for a,b in conexoes:
    g.connectVertices(dicionario_indice_vertices.get(a),dicionario_indice_vertices.get(b))

#rodar vertex color no grafo
array_cores = [0] * len(polygon)
g.vertexColor(3,array_cores,0) #no escopo, array de cores é modificado, logo pode ser usado em seguida

vertex_colors = {}
#se vértice preto, coloração errada
cores = ['black', 'purple', 'yellow', 'blue', 'black']

for i, vertex in enumerate(polygon):
    vertex_colors[vertex] = cores[array_cores[i]]

print(vertex_colors)

{(11.738629630766809, 7.3216515351086855): 'purple', (11.860933776944876, 7.723879873752594): 'yellow', (7.699581298045814, 9.777790376916528): 'blue', (5.43849278613925, 10.247327791526914): 'yellow', (7.7646241541951895, 7.545410039834678): 'purple', (10.59512207005173, 7.4548459416255355): 'blue', (8.793638563714921, 4.844651281833649): 'yellow', (3.3427570620551705, 10.970680316910148): 'blue', (3.257169222459197, 10.20868988148868): 'purple', (-0.1532835876569152, 8.385072126053274): 'blue', (-2.0294700264930725, 3.1217468148097396): 'purple', (-2.0232564210891724, 5.403068688698113): 'yellow', (-2.719149355776608, 5.188982569612563): 'blue', (-2.0808586115017533, 1.9411433571949601): 'yellow', (2.4004607666283846, 4.195138857699931): 'purple', (2.6442698864266276, 1.7809057272970676): 'blue', (5.08361873499883, 1.8030554465949535): 'purple', (11.187730520032346, 5.569734688848257): 'blue', (14.40893903374672, 2.865135650150478): 'yellow', (14.494080721400678, 9.34415570832789): '

In [8]:
fig = go.Figure()

x, y = zip(*polygon)

step_index = 1
x_cumulative = ()
y_cumulative = ()

for triangle in triangulos_plot:
    tx, ty = zip(*triangle)

    if triangle in triangles:
        x_cumulative += tx
        y_cumulative += ty
    step_index = step_index + 1

fig.add_trace(go.Scatter(visible=True,
                                x=x + (x[0],) + x_cumulative + (tx[0],),
                                y=y + (y[0],) + y_cumulative + (ty[0],),
                                mode='lines',
                                fill='none',
                                name='Polígono Triangulado',
                                showlegend=False))

for vertex, color in vertex_colors.items():
    fig.add_trace(go.Scatter(visible=False,
                              x=[vertex[0]],
                              y=[vertex[1]],
                              mode='markers',
                              marker=dict(color=color, size=20),
                              name=color + str(step_index),
                              showlegend=False))
    step_index = step_index + 1

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
            {"title": "Passo: " + str(i)}],
    )
    step["args"][0]["visible"][i] = True

    step["args"][0]["visible"][0] = True
    for j in range(1,i): #deixa cada vértice colorido visível ao mesmo tempo
        step["args"][0]["visible"][j] = True

    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": ""},
    steps=steps
)]

fig.update_layout(title='Coloração dos vértices',
                  xaxis_title='X',
                  yaxis_title='Y',
                  sliders=sliders)

fig.show()

Por fim temos todos os passos, desde a triangulação até a coloração dos vértices

In [9]:

fig = go.Figure()

x, y = zip(*polygon)
fig.add_trace(go.Scatter(visible=True,
                                x=x + (x[0],),
                                y=y + (y[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='blue'),
                                name='Polígono',
                                showlegend=True))

step_index = 1
x_cumulative = ()
y_cumulative = ()

for triangle in triangulos_plot:
    tx, ty = zip(*triangle)

    #se é um triangulo correto
    if triangle in triangles:
        x_cumulative += tx
        y_cumulative += ty
        fig.add_trace(go.Scatter(visible=False,
                                x=x_cumulative + (tx[0],),
                                y=y_cumulative + (ty[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='green'),
                                name='Polígono Triangulado',
                                showlegend=True))
    else:
        fig.add_trace(go.Scatter(visible=False,
                                x=x + (x[0],) + tx + (tx[0],),
                                y=y + (y[0],) + ty + (ty[0],),
                                mode='lines',
                                fill='none',
                                line=dict(color='red',dash='dot'),
                                name='Vértice dentro do Triangulo',
                                showlegend=True))
    step_index = step_index + 1



triangulacao_completa = step_index-1

for vertex, color in vertex_colors.items():
    fig.add_trace(go.Scatter(visible=False,
                              x=[vertex[0]],
                              y=[vertex[1]],
                              mode='markers',
                              marker=dict(color=color, size=20),
                              name=color + str(step_index),
                              showlegend=False))
    step_index = step_index + 1

steps = []
i = 0
while i < len(fig.data):
    pular2 = False

    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
            {"title": "Passo: " + str(i)}],
    )
    step["args"][0]["visible"][i] = True
    step["args"][0]["visible"][0] = True #Polígono inicial sempre visível

    if i >= triangulacao_completa: #triangulação completa sempre visivel depois de terminar
        step["args"][0]["visible"][triangulacao_completa] = True
        for j in range(triangulacao_completa+1,i): #deixa cada vértice colorido visível ao mesmo tempo
            step["args"][0]["visible"][j] = True

    steps.append(step)
    i += 1

sliders = [dict(
    active=0,
    currentvalue={"prefix": ""},
    steps=steps
)]

fig.update_layout(title='Triangulação por Corte de Orelhas e Coloração dos vértices',
                  xaxis_title='X',
                  yaxis_title='Y',
                  sliders=sliders)

fig.show()


Agora como é de nosso interesse saber quantos guardas são necessários para vigiar a galeria de arte (Problema da Galeria de Arte), fazemos a contagem de quantas vezes a mesma cor ocorre nos vértices e pegamos a menor.

In [10]:
contagem = {'blue': 0, 'purple': 0, 'yellow': 0}

for cor in vertex_colors.values():
    if cor in contagem:
        contagem[cor] = contagem[cor] + 1


print("São necessários ", min(contagem.values()), " guardas para vigiar a galeria de arte.")

São necessários  6  guardas para vigiar a galeria de arte.
